In [1]:
import pandas as pd
import altair as alt
import os

In [4]:
room_ids = set([x.split("_")[0] for x in os.listdir("data-bms")])
room_ids = list(filter(lambda x: "." not in x, room_ids))
room_ids

['52', '520', '522', '524']

In [7]:
def get_room_df(room_id):
    temp_act_df = pd.read_csv(f"data-bms/{room_id}_temp_act.csv", index_col=0)
    temp_target_df = pd.read_csv(f"data-bms/{room_id}_temp_target.csv", index_col=0)
    temp_act_target = pd.merge(temp_act_df, temp_target_df, on="time")
    presence_df = pd.read_csv(f"data-bms/{room_id}_presence.csv", index_col=0)
    temp_act_target_presence = pd.merge(temp_act_target, presence_df, on="time")
    temp_act_target_presence = temp_act_target_presence.rename({"state_x": "temp_act", 
        "state_y": "temp_target", "state": "presence"}, axis="columns")
    valve_df = pd.read_csv(f"data-bms/{room_id}_valve.csv", index_col=0)
    room_df = pd.merge(valve_df, temp_act_target_presence, on="time").rename({"state": "valve"}, axis="columns")
    room_df["room_id"] = room_id
    return room_df
    

In [8]:
room_df = pd.concat([get_room_df(room) for room in room_ids])
room_df

,time,valve,temp_act,temp_target,presence,room_id
0,2022-12-23 14:00:00+00:00,20.0,22.9,23.0,30.0,52
1,2022-12-23 15:00:00+00:00,20.0,22.7,23.0,30.0,52
2,2022-12-23 16:00:00+00:00,20.0,22.7,23.0,30.0,52
3,2022-12-23 17:00:00+00:00,17.0,22.3,21.0,0.0,52
4,2022-12-23 18:00:00+00:00,20.0,22.2,23.0,8.0,52
...,...,...,...,...,...,...
266,2023-01-03 16:00:00+00:00,18.0,21.7,21.0,0.0,524
267,2023-01-03 17:00:00+00:00,18.0,21.1,21.0,0.0,524
268,2023-01-03 18:00:00+00:00,18.0,21.3,21.0,0.0,524
269,2023-01-03 19:00:00+00:00,20.0,20.9,21.0,0.0,524


In [10]:
weather_df = pd.read_csv("data-bms/weather.csv")
room_df.time = pd.to_datetime(room_df.time).dt.strftime('%Y-%m-%d %H:%M:%S')
weather_df.time = pd.to_datetime(weather_df.time).dt.strftime('%Y-%m-%d %H:%M:%S')
pd.merge(room_df, weather_df, on="time")


,time,valve,temp_act,temp_target,presence,room_id,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2022-12-23 14:00:00,20.0,22.9,23.0,30.0,52,7.4,5.6,88.0,0.0,NaN,230.0,14.0,NaN,1010.2,NaN,3.0
1,2022-12-23 14:00:00,20.0,20.1,23.0,30.0,522,7.4,5.6,88.0,0.0,NaN,230.0,14.0,NaN,1010.2,NaN,3.0
2,2022-12-23 14:00:00,20.0,21.0,21.0,0.0,524,7.4,5.6,88.0,0.0,NaN,230.0,14.0,NaN,1010.2,NaN,3.0
3,2022-12-23 15:00:00,20.0,22.7,23.0,30.0,52,7.1,5.6,90.0,0.0,NaN,226.0,13.3,NaN,1010.2,NaN,3.0
4,2022-12-23 15:00:00,20.0,20.4,21.0,0.0,520,7.1,5.6,90.0,0.0,NaN,226.0,13.3,NaN,1010.2,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,2023-01-03 19:00:00,20.0,20.9,21.0,0.0,524,6.5,5.5,93.0,0.0,NaN,284.0,15.5,NaN,1028.5,NaN,3.0
1079,2023-01-03 20:00:00,18.0,22.2,21.0,0.0,52,6.4,5.1,91.0,0.0,NaN,290.0,15.1,NaN,1028.7,NaN,3.0
1080,2023-01-03 20:00:00,20.0,21.0,21.0,0.0,520,6.4,5.1,91.0,0.0,NaN,290.0,15.1,NaN,1028.7,NaN,3.0
1081,2023-01-03 20:00:00,20.0,20.7,21.0,0.0,522,6.4,5.1,91.0,0.0,NaN,290.0,15.1,NaN,1028.7,NaN,3.0
